In [1]:
import numpy as np
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes 
import folium 
print('Imports complete.')

/home/jupyterlab/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/jupyterlab/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/jupyterlab/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/jupyterlab/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/jupyterlab/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/jupyterlab/conda/lib/python3.6/importli

Solving environment: done

# All requested packages already installed.

Imports complete.


In [2]:
import requests
page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
page.status_code

200

In [3]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content,'html.parser')
My_table = soup.find('table',{'class':'wikitable sortable'})
links = My_table.findAll('td')
column_names = ['Postal Code','Borough', 'Neighborhood'] 
neighborhoods = pd.DataFrame(columns=column_names)

In [4]:
column_names = ['Postal Code','Borough', 'Neighborhood'] 
neighborhoods = pd.DataFrame(columns=column_names)

In [5]:
for i in range(int(len(links)/3)):
     
   Postcode=links[i*3:3*(i+1)][0].get_text()
   Borough=links[i*3:3*(i+1)][1].get_text()
   Neighborhood=links[i*3:3*(i+1)][2].get_text()[:-1]


   if Borough!='Not assigned':
        
    Neighborhood=Borough if Neighborhood =='Not assigned' else Neighborhood
    
    neighborhoods = neighborhoods.append({'Postal Code': Postcode,
                                          'Borough': Borough,
                                          'Neighborhood': Neighborhood}, ignore_index=True)

In [6]:
print(neighborhoods.shape)
neighborhoods.head(21)

(212, 3)


,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [7]:
neigh=neighborhoods.groupby(['Postal Code','Borough'])
df = neigh['Neighborhood'].unique()
df=df.reset_index()
print(df.shape)
df.head(21)

(103, 3)


,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"[Rouge, Malvern]"
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]"
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]"
3,M1G,Scarborough,[Woburn]
4,M1H,Scarborough,[Cedarbrae]
5,M1J,Scarborough,[Scarborough Village]
6,M1K,Scarborough,"[East Birchmount Park, Ionview, Kennedy Park]"
7,M1L,Scarborough,"[Clairlea, Golden Mile, Oakridge]"
8,M1M,Scarborough,"[Cliffcrest, Cliffside, Scarborough Village West]"
9,M1N,Scarborough,"[Birch Cliff, Cliffside West]"


In [8]:
geodata_df = pd.read_csv('Geospatial_Coordinates.csv')
print(geodata_df.shape)
geodata_df.head(21)

(103, 3)


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [9]:
dfg = pd.merge(df, geodata_df, on="Postal Code",how='left')
dfg.head(21)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"[Rouge, Malvern]",43.806686,-79.194353
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]",43.784535,-79.160497
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]",43.763573,-79.188711
3,M1G,Scarborough,[Woburn],43.770992,-79.216917
4,M1H,Scarborough,[Cedarbrae],43.773136,-79.239476
5,M1J,Scarborough,[Scarborough Village],43.744734,-79.239476
6,M1K,Scarborough,"[East Birchmount Park, Ionview, Kennedy Park]",43.727929,-79.262029
7,M1L,Scarborough,"[Clairlea, Golden Mile, Oakridge]",43.711112,-79.284577
8,M1M,Scarborough,"[Cliffcrest, Cliffside, Scarborough Village West]",43.716316,-79.239476
9,M1N,Scarborough,"[Birch Cliff, Cliffside West]",43.692657,-79.264848


In [10]:
Toronto_data = dfg[dfg['Borough'].str.contains('Toronto', regex=False, case=False, na=False)].reset_index(drop=True)
print(Toronto_data.shape)
Toronto_data.head(21)

(38, 5)


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,[The Beaches],43.676357,-79.293031
1,M4K,East Toronto,"[The Danforth West, Riverdale]",43.679557,-79.352188
2,M4L,East Toronto,"[The Beaches West, India Bazaar]",43.668999,-79.315572
3,M4M,East Toronto,[Studio District],43.659526,-79.340923
4,M4N,Central Toronto,[Lawrence Park],43.728020,-79.388790
5,M4P,Central Toronto,[Davisville North],43.712751,-79.390197
6,M4R,Central Toronto,[North Toronto West],43.715383,-79.405678
7,M4S,Central Toronto,[Davisville],43.704324,-79.388790
8,M4T,Central Toronto,"[Moore Park, Summerhill East]",43.689574,-79.383160
9,M4V,Central Toronto,"[Deer Park, Forest Hill SE, Rathnelly, South H...",43.686412,-79.400049


# Foursquare Creds

In [11]:
CLIENT_ID = 'SMNETOO2A5RQJK5JGSRAPJIV5444JMP2WJIXHTFLJ0GZLPQS' 
CLIENT_SECRET = 'QHDB4QGRBDNEQNG5PUPIXYMWVUUHYCLRYQPHQUYZVFHRZVTQ' 
VERSION = '20180605'
LIMIT = 100 
radius = 500 

# Function call for venues

In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for names, lat, lng in zip(names, latitudes, longitudes):
        
       for name in names:
        
            
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
    return(nearby_venues)

# Create dataframe for Toronto Venues

In [13]:
Toronto_venues = getNearbyVenues(names=Toronto_data['Neighborhood'],
                                   latitudes=Toronto_data['Latitude'],
                                   longitudes=Toronto_data['Longitude']
                                  )

In [16]:
print(Toronto_venues.shape)
Toronto_venues.head(101)

(3314, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
1,The Beaches,43.676357,-79.293031,Starbucks,43.678798,-79.298045,Coffee Shop
2,The Beaches,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,The Danforth West,43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant
5,The Danforth West,43.679557,-79.352188,Dolce Gelato,43.677773,-79.351187,Ice Cream Shop
6,The Danforth West,43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop
7,The Danforth West,43.679557,-79.352188,Messini Authentic Gyros,43.677827,-79.350569,Greek Restaurant
8,The Danforth West,43.679557,-79.352188,Cafe Fiorentina,43.677743,-79.350115,Italian Restaurant
9,The Danforth West,43.679557,-79.352188,Mezes,43.677962,-79.350196,Greek Restaurant


# Determine unique catagories

In [15]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 237 uniques categories.
